In [46]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [47]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms, datasets

try:
    from torchinfo import summary
except:
    !pip install -q torchinfo
    from torchinfo import summary

In [48]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [49]:
import requests
import zipfile

from pathlib import Path

data_path=Path("data/")
image_path=data_path / "pizza_steak_sushi"

image_path.mkdir(parents=True, exist_ok=True)
    
with open(data_path / "pizza_steak_sushi.zip","wb") as f:
    request=requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    f.write(request.content)
        
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip","r")as zip_ref:
    zip_ref.extractall(image_path)
    
os.remove(data_path / "pizza_steak_sushi.zip")

In [50]:
train_dir=image_path / "train"
test_dir=image_path / "test"

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

In [51]:
data_transform=transforms.Compose([
    
    #resize the images to 64x64
    transforms.Resize(size=(64,64)),
    
    # flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5), # p= probability of flip
    
    # turn the image into a torch.Tensor
    transforms.ToTensor(), # this aslo convert all pixel value from 0to 255 to be between 0 and 1
    
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [52]:
# use ImageFolder to create datasets
train_data=datasets.ImageFolder(root=train_dir, # target folderof image
                                transform=data_transform, # 
                                target_transform=None)# transforms to perform on label (if necesary)
test_data=datasets.ImageFolder(root=test_dir,
                               transform=data_transform)

In [53]:
# tur n train adn ttest datasets inot dataloaders
from torch.utils.data import DataLoader

train_dataloader=DataLoader(dataset=train_data,
                            batch_size=1,
                            num_workers=1, # how many suprocess to usefor data loading?
                            shuffle=True)
test_dataloader=DataLoader(dataset=test_data,
                           batch_size=1,
                           num_workers=1,
                           shuffle=False)
train_dataloader,test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x794e7c4b6500>,
 <torch.utils.data.dataloader.DataLoader at 0x794e7c4b6230>)

In [54]:
weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT
weights

EfficientNet_B0_Weights.IMAGENET1K_V1

In [55]:
auto_transforms=weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [56]:
train_data=datasets.ImageFolder(root=train_dir, # target folderof image
                                transform=data_transform, # 
                                target_transform=None)# transforms to perform on label (if necesary)
test_data=datasets.ImageFolder(root=test_dir,
                               transform=data_transform)

In [57]:
train_dataloader=DataLoader(dataset=train_data,
                            batch_size=1,
                            num_workers=1, # how many suprocess to usefor data loading?
                            shuffle=True)
test_dataloader=DataLoader(dataset=test_data,
                           batch_size=1,
                           num_workers=1,
                           shuffle=False)
train_dataloader,test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x794e7c433dc0>,
 <torch.utils.data.dataloader.DataLoader at 0x794e7c432fb0>)

In [58]:
weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT
model=torchvision.models.efficientnet_b0(weights).to(device)
model

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [59]:
summary(model=model,
        input_size=(32, 3, 224, 224),
        col_names=["input_size","output_size","num_params","trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

In [60]:
# freeze all base layers in the "features" section of the model
for param in model.features.parameters():
    param.requires_grad=False 

In [61]:
torch.manual_seed(29)

torch.cuda.manual_seed(29)

output_shape=3

model.classifier=torch.nn.Sequential(
    torch.nn.Dropout(p=0.2,inplace=True),
    torch.nn.Linear(in_features=1280,
                    out_features=output_shape,
                    bias=True)
).to(device)

In [62]:
summary(model=model,
        input_size=(32, 3, 224, 224),
        col_names=["input_size","output_size","num_params","trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [63]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),
                          lr=0.01)

In [64]:
def train_step(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer):
    model.train()
    
    train_loss, train_acc=0,0
    
    for batch, (X,y) in enumerate(dataloader):
        X,y=X.to(device), y.to(device)
        
        y_pred=model(X)
        
        loss=loss_fn(y_pred,y)
        train_loss+=loss.item()
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        y_pred_class=torch.argmax(torch.softmax(y_pred,dim=1),dim=1)
        train_acc+=(y_pred_class==y).sum().item()/len(y_pred)
    
    # adjust metrics to get average loss and accuracy pre batch
    train_loss=train_loss / len(dataloader)
    train_acc=train_acc/ len(dataloader)
    
    return train_loss, train_acc

In [65]:
def test_step(model:torch.nn.Module,
              dataloader:torch.utils.data.DataLoader,
              loss_fn:torch.nn.Module):
    
    model.eval()
    
    test_loss, test_acc=0,0
    
    with torch.inference_mode():
        for batch, (X,y) in enumerate(dataloader):
            X,y=X.to(device), y.to(device)
            
            test_pred_logits=model(X)
            
            loss=loss_fn(test_pred_logits,y)
            test_loss+=loss.item()
            
            test_pred_labels=test_pred_logits.argmax(dim=1)
            test_acc+=((test_pred_labels==y).sum().item() / len(test_pred_labels))
            
    test_loss/=len(dataloader)
    test_acc/=len(dataloader)
    return test_loss, test_acc

In [66]:
from tqdm.auto import tqdm
from torch.utils.tensorboard import SummaryWriter

def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn.Module,
          epochs:int=5):
    results={"train_loss":[],
             "train_acc":[],
             "test_loss":[],
             "test_acc":[]}
    
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc=train_step(model=model,
                                         dataloader=train_dataloader,
                                         loss_fn=loss_fn,
                                         optimizer=optimizer)
        test_loss, test_acc=test_step(model=model,
                                      dataloader=test_dataloader,
                                      loss_fn=loss_fn)
        
        print(
            f"{epoch+1} | " 
            f"train_loss: {train_loss} | "
            f"train_acc: {train_acc} | "            
            f"test_loss: {test_loss} | "
            f"test_acc: {test_acc} "
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        # Experimint tracking
        writer=SummaryWriter()
        
        # add loss results to summary writer
        writer.add_scalars(main_tag="Loss",
                           tag_scalar_dict={"train_loss":train_loss,
                                           "test_loss":test_loss},
                           global_step=epochs
                        )
        # add accuracy results to SummaryWriter
        writer.add_scalars(main_tag="Accuracy",
                           tag_scalar_dict={"train_acc":train_acc,
                                            "test_acc":test_acc},
                           global_step=epochs
                        )
        
        # track the pytorch model architeccture
        writer.add_graph(model=model,
                         input_to_model=torch.randn(32,3,224,224).to(device))
    writer.close()
        
    return results

In [67]:
torch.manual_seed(29)
torch.cuda.manual_seed(29)

results=train(model=model,
              train_dataloader=train_dataloader,
              test_dataloader=test_dataloader,
              optimizer=optimizer,
              loss_fn=loss_fn,
              epochs=5,
              )

  0%|          | 0/5 [00:00<?, ?it/s]

1 | train_loss: 1.7380478501588934 | train_acc: 0.3422222222222222 | test_loss: 24.731089765882963 | test_acc: 0.30666666666666664 
2 | train_loss: 2.2342430224041974 | train_acc: 0.29333333333333333 | test_loss: 14.397330179707447 | test_acc: 0.3333333333333333 
3 | train_loss: 1.8697436951571662 | train_acc: 0.38666666666666666 | test_loss: 18.58480136554159 | test_acc: 0.3333333333333333 
4 | train_loss: 1.821028441426396 | train_acc: 0.4444444444444444 | test_loss: 17.84432128576974 | test_acc: 0.3333333333333333 
5 | train_loss: 1.9106864041299558 | train_acc: 0.4577777777777778 | test_loss: 20.188492110509777 | test_acc: 0.29333333333333333 


In [68]:
results

{'train_loss': [1.7380478501588934,
  2.2342430224041974,
  1.8697436951571662,
  1.821028441426396,
  1.9106864041299558],
 'train_acc': [0.3422222222222222,
  0.29333333333333333,
  0.38666666666666666,
  0.4444444444444444,
  0.4577777777777778],
 'test_loss': [24.731089765882963,
  14.397330179707447,
  18.58480136554159,
  17.84432128576974,
  20.188492110509777],
 'test_acc': [0.30666666666666664,
  0.3333333333333333,
  0.3333333333333333,
  0.3333333333333333,
  0.29333333333333333]}

In [69]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 1265), started 0:30:50 ago. (Use '!kill 1265' to kill it.)

In [70]:
def create_writer(experiment_name:str,
                  model_name:str,
                  extra:str=None)-> torch.utils.tensorboard.writer.SummaryWriter():
    
    from datetime import datetime
    import os
    
    timestamp=datetime.now().strftime("Y-%m-%d")
    
    if extra:
        log_dir=os.path.join("runs",timestamp, experiment_name, model_name, extra)
    else:
        log_dir=os.path.join("runs",timestamp, experiment_name, model_name)
        
    return SummaryWriter(log_dir=log_dir)

In [71]:
example_writer=create_writer(experiment_name="data_10_percent",
                             model_name="efficienb0",
                             extra="5_epochs")

In [72]:
from typing import Dict, List
from torch.utils.tensorboard import SummaryWriter

def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn.Module,
          writer:torch.utils.tensorboard.writer.SummaryWriter,
          epochs:int=5)-> Dict[str, List]:
    results={"train_loss":[],
             "train_acc":[],
             "test_loss":[],
             "test_acc":[]}
    
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc=train_step(model=model,
                                         dataloader=train_dataloader,
                                         loss_fn=loss_fn,
                                         optimizer=optimizer)
        test_loss, test_acc=test_step(model=model,
                                      dataloader=test_dataloader,
                                      loss_fn=loss_fn)
        
        print(
            f"{epoch+1} | " 
            f"train_loss: {train_loss} | "
            f"train_acc: {train_acc} | "            
            f"test_loss: {test_loss} | "
            f"test_acc: {test_acc} "
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        
        # Experimint tracking
        
        # see if ther is a writer, if so , log it
        # add loss results to summary writer
        if writer:
            writer.add_scalars(main_tag="Loss",
                               tag_scalar_dict={"train_loss":train_loss,
                                               "test_loss":test_loss},
                               global_step=epochs
                            )
            # add accuracy results to SummaryWriter
            writer.add_scalars(main_tag="Accuracy",
                               tag_scalar_dict={"train_acc":train_acc,
                                                "test_acc":test_acc},
                               global_step=epochs
                            )
        
            writer.close()
        
    return results

In [73]:
import os 
import zipfile

from pathlib import Path

def download_data(source:str,
                  destination:str,
                  remove_source:bool=True) -> Path:
                  
    data_path=Path("data/")
    image_path=data_path / destination
                  
    if image_path.is_dir()==False:

        image_path.mkdir(parents=True, exist_ok=True)

        with open(data_path / "pizza_steak_sushi.zip","wb") as f:
            request=requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
            f.write(request.content)

        with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip","r")as zip_ref:
            zip_ref.extractall(image_path)
        
        if remove_source:
            os.remove(data_path / "pizza_steak_sushi.zip")
    return image_path

In [74]:
# Download 10 percent and 20 percent training data (if necessary)
data_10_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
                                     destination="pizza_steak_sushi")

data_20_percent_path = download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
                                     destination="pizza_steak_sushi_20_percent")

In [82]:
train_dir_10_percent=data_10_percent_path / "train"
train_dir_20_percent=data_20_percent_path / "train"
test_dir=image_path / "test"

In [83]:
simple_transform=transforms.Compose([
    
    #resize the images to 64x64
    transforms.Resize(size=(64,64)),
    
    # flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5), # p= probability of flip
    
    # turn the image into a torch.Tensor
    transforms.ToTensor(), # this aslo convert all pixel value from 0to 255 to be between 0 and 1
    
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [86]:
import torchvision.datasets as datasets
# use ImageFolder to create datasets
train_data_10_percent=datasets.ImageFolder(root=train_dir_10_percent, # target folderof image
                                transform=data_transform, # 
                                target_transform=None)# transforms to perform on label (if necesary)
train_data_20_percent=datasets.ImageFolder(root=train_dir_20_percent, # target folderof image
                                transform=data_transform, # 
                                target_transform=None)# transforms to perform on label (if necesary)
test_data=datasets.ImageFolder(root=test_dir,
                               transform=data_transform)

In [88]:
train_dataloader_10_percent=DataLoader(dataset=train_data_10_percent,
                            batch_size=32,
                            num_workers=1, # how many suprocess to usefor data loading?
                            shuffle=True)
train_dataloader_20_percent=DataLoader(dataset=train_data_20_percent,
                            batch_size=32,
                            num_workers=1, # how many suprocess to usefor data loading?
                            shuffle=True)

test_dataloader=DataLoader(dataset=test_data,
                           batch_size=32,
                           num_workers=1,
                           shuffle=False)
train_dataloader_10_percent, train_dataloader_10_percent, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x794efd469570>,
 <torch.utils.data.dataloader.DataLoader at 0x794efd469540>)

In [95]:
import torchvision
from torchinfo import summary

effnetb2_weights=torchvision.models.EfficientNet_B2_Weights.DEFAULT
effnetb2=torchvision.models.efficientnet_b2(weights=effnetb2_weights)

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 113MB/s] 


In [102]:
import torchvision
from torch import nn

OUT_FEATURES=3

def create_effnetb0():
    
    effnetb2_weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT
    effnetb2=torchvision.models.efficientnet_b0(weights=effnetb2_weights)
    
    for param in model.features.parameters():
        param.requires_grad=False
        
    torch.manual_seed(29)
    torch.cuda.manual_seed(29)
    
    model.classifier=nn.Sequential(
        nn.Dropout(p=0.2),
        nn.Linear(in_features=1280, 
                  out_features=OUT_FEATURES)
    ).to(device)
    
    model.name="effnetb0"
    
    return model

In [103]:
def create_effnetb2():
    
    effnetb2_weights=torchvision.models.EfficientNet_B2_Weights.DEFAULT
    effnetb2=torchvision.models.efficientnet_b2(weights=effnetb2_weights)
    
    for param in model.features.parameters():
        param.requires_grad=False
        
    torch.manual_seed(29)
    torch.cuda.manual_seed(29)
    
    model.classifier=nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features=1408, 
                  out_features=OUT_FEATURES)
    ).to(device)
    
    model.name="effnetb2"
    
    return model

In [105]:
effnetb0=create_effnetb0()

In [107]:
effnetb2=create_effnetb2()

In [110]:
summary(model=effnetb0,
        input_size=(32, 3, 224, 224),
        col_names=["input_size","output_size","num_params","trainable"],
        col_width=20,
        row_settings=["var_names"]
)

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Sequential: 1, Conv2dNormActivation: 2, Conv2d: 3, BatchNorm2d: 3, SiLU: 3, Sequential: 2, MBConv: 3, Sequential: 4, Conv2dNormActivation: 5, Conv2d: 6, BatchNorm2d: 6, SiLU: 6, SqueezeExcitation: 5, AdaptiveAvgPool2d: 6, Conv2d: 6, SiLU: 6, Conv2d: 6, Sigmoid: 6, Conv2dNormActivation: 5, Conv2d: 6, BatchNorm2d: 6, Sequential: 2, MBConv: 3, Sequential: 4, Conv2dNormActivation: 5, Conv2d: 6, BatchNorm2d: 6, SiLU: 6, Conv2dNormActivation: 5, Conv2d: 6, BatchNorm2d: 6, SiLU: 6, SqueezeExcitation: 5, AdaptiveAvgPool2d: 6, Conv2d: 6, SiLU: 6, Conv2d: 6, Sigmoid: 6, Conv2dNormActivation: 5, Conv2d: 6, BatchNorm2d: 6, MBConv: 3, Sequential: 4, Conv2dNormActivation: 5, Conv2d: 6, BatchNorm2d: 6, SiLU: 6, Conv2dNormActivation: 5, Conv2d: 6, BatchNorm2d: 6, SiLU: 6, SqueezeExcitation: 5, AdaptiveAvgPool2d: 6, Conv2d: 6, SiLU: 6, Conv2d: 6, Sigmoid: 6, Conv2dNormActivation: 5, Conv2d: 6, BatchN

In [111]:
num_epochs=[5,10]

models=["effnetb0","effnetb2"]

train_dataloaders={"data_10_percent":train_dataloader_10_percent,
                   "data_20_percent":train_data_20_percent}

In [115]:
%%time

experiment_number=0

for dataloader_name, train_dataloader in train_dataloaders.items():
    for epochs in num_epochs:
        for model_name in models:
            experiment_number+=1
        
            print(experiment_number)
            print(model_name)
            print(dataloader_name)
            print(epochs)
            
            if model_name=="effnetb0":
                model=create_effnetb0()
            else:
                model=create_effnetb2()
            
            loss_fn=nn.CrossEntropyLoss()
            optimizer=torch.optim.Adam(params=model.parameters(),lr=0.01)
            
            train(model=model,
                  train_dataloader=train_dataloader,
                  test_dataloader=test_dataloader,
                  optimizer=optimizer,
                  loss_fn=loss_fn,
                  epochs=epochs,
                  writer=create_writer(experiment_name=dataloader_name,
                                       model_name=model_name,
                                       extra=f"epochs: {epochs}"))        

1
effnetb0
data_10_percent
5


  0%|          | 0/5 [00:00<?, ?it/s]

1 | train_loss: 1.1265328451991081 | train_acc: 0.421875 | test_loss: 1.0675922632217407 | test_acc: 0.49526515151515155 
2 | train_loss: 0.8807534277439117 | train_acc: 0.5859375 | test_loss: 1.014660159746806 | test_acc: 0.537878787878788 
3 | train_loss: 0.766521580517292 | train_acc: 0.63671875 | test_loss: 0.840299387772878 | test_acc: 0.6590909090909091 
4 | train_loss: 0.8942751996219158 | train_acc: 0.66015625 | test_loss: 1.1021015445391338 | test_acc: 0.6410984848484849 
5 | train_loss: 0.7340848110616207 | train_acc: 0.6796875 | test_loss: 0.8133730888366699 | test_acc: 0.7509469696969697 
2
effnetb2
data_10_percent
5


  0%|          | 0/5 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x1280 and 1408x3)